# Базовий фреймворк для задач задоволення обмежень

Базова модель задачі задоволення обмежень:

In [7]:

from typing import Generic, TypeVar, Dict, List, Optional
from abc import ABC, abstractmethod

V = TypeVar('V')
D = TypeVar('D')


class Constraint(Generic[V, D], ABC):
    def __init__(self, variables: List[V]) -> None:
        self.variables = variables

    @abstractmethod
    def satisfied(self, assignment: Dict[V, D]) -> bool:
        ...

class CSP(Generic[V, D]):
    def __init__(self, variables: List[V], domains: Dict[V, List[D]]) -> None:
        self.variables: List[V] = variables # variables to be constrained
        self.domains: Dict[V, List[D]] = domains # domain of each variable
        self.constraints: Dict[V, List[Constraint[V, D]]] = {}
        for variable in self.variables:
            self.constraints[variable] = []
            if variable not in self.domains:
                raise LookupError("Every variable should have a domain assigned to it.")

    def add_constraint(self, constraint: Constraint[V, D]) -> None:
        for variable in constraint.variables:
            if variable not in self.variables:
                raise LookupError("Variable in constraint not in CSP")
            else:
                self.constraints[variable].append(constraint)

    def consistent(self, variable: V, assignment: Dict[V, D]) -> bool:
        for constraint in self.constraints[variable]:
            if not constraint.satisfied(assignment):
                return False
        return True
    def backtracking_search(self, assignment: Dict[V, D] = {}) -> Optional[Dict[V, D]]:
        if len(assignment) == len(self.variables):
            return assignment

        unassigned: List[V] = [v for v in self.variables if v not in assignment]

        first: V = unassigned[0]
        for value in self.domains[first]:
            local_assignment = assignment.copy()
            local_assignment[first] = value
            if self.consistent(first, local_assignment):
                result: Optional[Dict[V, D]] = self.backtracking_search(local_assignment)
                if result is not None:
                    return result
        return None


Базовий метод - пошук з поверненнями(backtracking):

In [8]:
import random
from typing import Tuple

class CSPWithMCV(CSP[V, D]):
    def select_unassigned_variable(self, assignment: Dict[V, D]) -> V:
        unassigned: List[V] = [v for v in self.variables if v not in assignment]
        return min(unassigned, key=lambda var: len(self.domains[var]))

    def backtracking_search_with_mcv(self, assignment: Dict[V, D] = {}) -> Optional[Dict[V, D]]:
        if len(assignment) == len(self.variables):
            return assignment

        var = self.select_unassigned_variable(assignment)
        for value in self.domains[var]:
            local_assignment = assignment.copy()
            local_assignment[var] = value
            if self.consistent(var, local_assignment):
                result: Optional[Dict[V, D]] = self.backtracking_search_with_mcv(local_assignment)
                if result is not None:
                    return result
        return None

def solve_n_queens_with_mcv(size: int):
    columns = list(range(size))
    domains = {col: list(range(size)) for col in columns}
    csp = CSPWithMCV(columns, domains)
    csp.add_constraint(QueensConstraint(columns))
    solution = csp.backtracking_search_with_mcv()
    return solution


# for size in [8, 12, 16]:
#     print(f"Solving {size}x{size} board with MCV...")
#     solution = solve_n_queens_with_mcv(size)
#     print(f"Solution for {size}x{size} board: {solution}")


Повертається перше знайдене рішення!

In [16]:
import random
from typing import List, Tuple

def fitness(assignment: List[int]) -> int:
    attacking_pairs = 0
    n = len(assignment)
    for i in range(n):
        for j in range(i + 1, n):
            if assignment[i] == assignment[j] or abs(assignment[i] - assignment[j]) == abs(i - j):
                attacking_pairs += 1
    return attacking_pairs

def initialize_population(pop_size: int, n: int) -> List[List[int]]:
    return [random.sample(range(n), n) for _ in range(pop_size)]

def select(population: List[List[int]], fitnesses: List[int], k: int) -> List[List[int]]:
    selected = random.choices(population, weights=[1/f for f in fitnesses], k=k)
    return selected

def crossover(parent1: List[int], parent2: List[int]) -> Tuple[List[int], List[int]]:
    n = len(parent1)
    crossover_point = random.randint(1, n - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutate(solution: List[int], mutation_rate: float) -> List[int]:
    n = len(solution)
    if random.random() < mutation_rate:
        i, j = random.sample(range(n), 2)
        solution[i], solution[j] = solution[j], solution[i]
    return solution

def genetic_algorithm(n: int, pop_size: int, generations: int, mutation_rate: float) -> List[int]:
    population = initialize_population(pop_size, n)
    best_solution = None
    best_fitness = float('inf')

    for generation in range(generations):
        fitnesses = [fitness(sol) for sol in population]
        best_idx = fitnesses.index(min(fitnesses))
        if fitnesses[best_idx] < best_fitness:
            best_fitness = fitnesses[best_idx]
            best_solution = population[best_idx]

        if best_fitness == 0:
            break

        selected = select(population, fitnesses, pop_size // 2)
        next_generation = []

        while len(next_generation) < pop_size:
            parent1, parent2 = random.sample(selected, 2)
            child1, child2 = crossover(parent1, parent2)
            next_generation.append(mutate(child1, mutation_rate))
            next_generation.append(mutate(child2, mutation_rate))

        population = next_generation

    return best_solution


# Задача з ферзями

Реалізація обмеження задачі:

In [10]:
class QueensConstraint(Constraint[int, int]):
    def __init__(self, columns: List[int]) -> None:
        super().__init__(columns)
        self.columns: List[int] = columns

    def satisfied(self, assignment: Dict[int, int]) -> bool:
        for q1c, q1r in assignment.items():
            for q2c in range(q1c + 1, len(self.columns) + 1):
                if q2c in assignment:
                    q2r: int = assignment[q2c]
                    if q1r == q2r:
                        return False
                    if abs(q1r - q2r) == abs(q1c - q2c):
                        return False
        return True

Рішення:

In [17]:
import time

# Function to compare all three algorithms
def compare_algorithms():
    for size in [8, 12, 16, 20]:
        print(f"\nSolving {size}x{size} board:")

        # Simple Backtracking
        start_time = time.time()
        simple_csp = CSP(list(range(size)), {i: list(range(size)) for i in range(size)})
        simple_csp.add_constraint(QueensConstraint(list(range(size))))
        simple_solution = simple_csp.backtracking_search()
        simple_time = time.time() - start_time
        print(f"Simple Backtracking Solution: {simple_solution} (Time: {simple_time:.4f} seconds)")

        # Backtracking with MCV
        start_time = time.time()
        mcv_solution = solve_n_queens_with_mcv(size)
        mcv_time = time.time() - start_time
        print(f"MCV Solution: {mcv_solution} (Time: {mcv_time:.4f} seconds)")

        # Genetic Algorithm
        start_time = time.time()
        ga_solution = genetic_algorithm(size, pop_size=100, generations=1000, mutation_rate=0.05)
        ga_time = time.time() - start_time
        ga_fitness = fitness(ga_solution)
        print(f"GA Solution: {ga_solution} (Fitness: {ga_fitness}, Time: {ga_time:.4f} seconds)")

# Run the comparison
compare_algorithms()



Solving 8x8 board:
Simple Backtracking Solution: {0: 0, 1: 4, 2: 7, 3: 5, 4: 2, 5: 6, 6: 1, 7: 3} (Time: 0.0108 seconds)
MCV Solution: {0: 0, 1: 4, 2: 7, 3: 5, 4: 2, 5: 6, 6: 1, 7: 3} (Time: 0.0094 seconds)
GA Solution: [3, 6, 0, 7, 4, 1, 5, 2] (Fitness: 0, Time: 0.0565 seconds)

Solving 12x12 board:
Simple Backtracking Solution: {0: 0, 1: 2, 2: 4, 3: 7, 4: 9, 5: 11, 6: 5, 7: 10, 8: 1, 9: 6, 10: 8, 11: 3} (Time: 0.0252 seconds)
MCV Solution: {0: 0, 1: 2, 2: 4, 3: 7, 4: 9, 5: 11, 6: 5, 7: 10, 8: 1, 9: 6, 10: 8, 11: 3} (Time: 0.0242 seconds)
GA Solution: [10, 6, 4, 2, 0, 11, 7, 1, 3, 9, 5, 8] (Fitness: 1, Time: 2.1862 seconds)

Solving 16x16 board:
Simple Backtracking Solution: {0: 0, 1: 2, 2: 4, 3: 1, 4: 12, 5: 8, 6: 13, 7: 11, 8: 14, 9: 5, 10: 15, 11: 6, 12: 3, 13: 10, 14: 7, 15: 9} (Time: 2.8198 seconds)
MCV Solution: {0: 0, 1: 2, 2: 4, 3: 1, 4: 12, 5: 8, 6: 13, 7: 11, 8: 14, 9: 5, 10: 15, 11: 6, 12: 3, 13: 10, 14: 7, 15: 9} (Time: 2.7632 seconds)
GA Solution: [8, 12, 0, 13, 3, 6, 11